In [1]:
from listennotes import podcast_api
import json 
import sys
import requests
from io import BytesIO
from PIL import Image
from PyQt6.QtCore import Qt
from PyQt6.QtWidgets import (QApplication, QCheckBox, QComboBox,  QDateEdit, QPushButton,
    QMainWindow,
    QVBoxLayout,
    QWidget, QLabel)
from PyQt6.QtGui import QIcon

api_key = open("../01_data/private/key.txt", "r").readline() # load api

with open("../01_data/examplefile.txt", "rt") as f: #load genre ids selected
    for line in f:
        genre_ids= line.split(',')

In [2]:
#fetch best podcasts in genre

def get_podcast(term):
    client = podcast_api.Client(api_key=api_key)      
    response = client.fetch_best_podcasts(genre_id=term, page=2, region='us')
    #print(response.json)
    return response.json()


In [3]:
#display best podcasts and select from list to prepare queue
#merge json for recommendation during drive time

json_list=[]
#for term in genre_ids:
response=get_podcast(genre_ids[0])
    #display in gui

json_list=response.get('podcasts')

pod_dict={}
pick=5 # allows to choose for 5 pods

i=0
#for each podcast store the title and image to add to combobox
for pod in json_list:
    i=i+1
    pod_dict[pod.get('title')]=pod.get('image')
    response = requests.get(pod.get('image'))
    img = Image.open(BytesIO(response.content))
    img.save("../03_output/"+pod.get('title')+".png", "PNG")
    if i==pick:
        break


In [4]:

# create a window to take in queue suggestions
class MainWindow(QMainWindow):
    def __init__(self):
        super().__init__()

        self.setWindowTitle("Create Queue")
        
        label = QLabel('Select a podcast for your queue:')
        
        layout = QVBoxLayout()
    
        combobox= QComboBox() # intitialize a combo box
        
        for pod in pod_dict:
            
            #icon=QIcon("../03_output/"+pod+".png")
            combobox.addItem(pod)
        
         # Connect signals
       
        combobox.currentTextChanged.connect(self.text_changed)
        
        confirm = QPushButton()
        confirm.setText('Confirm Selection')
        # confirm.setEnabled(False)
        confirm.clicked.connect(self.close)
  
        layout = QVBoxLayout()
        layout.addWidget(label)
        layout.addWidget(combobox)
        
        container = QWidget()
        container.setLayout(layout)
        layout.addWidget(confirm)

        self.setCentralWidget(container)
        
    def text_changed(self, s):
        return s

    def current_text_changed(self, s):
        return s

app = QApplication(sys.argv)
window = MainWindow()
window.show()


app.exec()

TypeError: text_changed() missing 2 required positional arguments: 'self' and 's'

: 